# 深層学習ノートブック-6 誤差逆伝播(Backpropagation)
誤差逆伝播をスクラッチで実装してみる   
* $l$番目の層における損失の誤差 
$$\delta^{[l]}=\frac{\partial{L}}{\partial{Z^{[l]}}}$$  
$$=\frac{\partial{L}}{\partial{A^{[l]}}} \bigodot \frac{\partial{A}}{\partial{Z^{[l]}}}$$
$$=(\delta^{[l+1]}\bm{W}^{[l+1]}) \bigodot \sigma'(\bm{Z^{[l]}})$$  

* パラメタの勾配
$$\frac{\partial{L}}{\partial{W^{[l]}}}=\delta^{[l]T}\bm{A}^{[l-1]}$$  
$$\frac{\partial{L}}{\partial{b^{[l]}}}=\sum_i\delta_i^{[l]}$$  

* パラメタの更新
$$\bm{W^{[l]}}=\bm{W^{[l]}}-\alpha\frac{\partial{L}}{\partial{W^{[l]}}}$$  
$$\bm{B^{[l]}}=\bm{B^{[l]}}-\alpha\frac{\partial{L}}{\partial{B^{[l]}}}$$  


※ReLUの導関数
\begin{equation}
\sigma^{'}(z)= \left \{
\begin{array}{l}
1　(if z > 0) \\
0　(otherwise)
\end{array}
\right.
\end{equation}

In [3]:
import torch
import torch.nn.functional as F  #pytorchの便利関数はFでimportすることが多い。
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# python debugerをインポート
import pdb

## ■ backward（逆伝播）の実装

In [36]:
# 線形変換部分(パラメタ)の勾配の算出関数。Aはl-1層目の値で他はl層目の値。
def linear_backward(A, W, b, Z):
     # pytorch.tensorの.grad属性と区別するために.grad_として属性を定義している
     W.grad_ = Z.grad_.T @ A
     # bの値が一つ変わると、全データ分Lへ影響を及ぼすので、行方向(データ数の方向)にZ.grad_を足す。
     b.grad_ = torch.sum(Z.grad_, dim=0)
     # l-1層目のAはl層目の誤差（=Z.grad_）とl層目のWから計算できる。
     A.grad_ = Z.grad_ @ W


# ReLU関数
def relu_backward(Z, A):
     # 上式の"l番目の層における損失の誤差"における2行目と3行目を組み合わせてdL/dZを算出している
     # A.grad_はdL/dA, (Z > 0).float()はReLu関数の偏微分に相当する。
     Z.grad_ = A.grad_ * ( Z > 0 ).float()

### 補足1
tensor * (tensor > 0)の計算について

In [5]:
a = torch.randn((2,3)) 
b = torch.randn((2,3)) 
print(a)
print(b)
print(b > 0)

tensor([[-2.4928,  0.5298,  2.3520],
        [-0.0545, -0.6147,  0.4214]])
tensor([[-0.5320,  2.0069,  1.3527],
        [-0.6340, -0.3005,  0.7240]])
tensor([[False,  True,  True],
        [False, False,  True]])


In [6]:
a * (b > 0)

tensor([[-0.0000, 0.5298, 2.3520],
        [-0.0000, -0.0000, 0.4214]])

このようにb > 0がTrueの要素は1、Falseの要素は0として計算される。

上記のように損失、パラメタの勾配をコードで表わすことが出来た。  
ではこれらの計算を実際に実行するにはどうすればよいか？  
当然、具体的な計算を行うためには上記の関数の入力であるA,W,b,Zを求める必要がある。  
すなわち順伝播(forward)の計算を行い、損失を計算してから逆伝播(backward)の計算を行うことになる。  

まず損失を求める関数を実装する。

In [7]:
# softmax関数と交差エントロピーの計算を同じ関数で実装する（pytorchでもこうなっている）
def softmax_cross_entropy(X, y_true):
    '''
    X: input tensor.行は各データ、列は各クラスを想定。
    y_true: tensor。One-Hot Encoding済みの正解ラベル。
    '''
    max_val = X.max(dim=1, keepdim=True).values
    # 各要素のe^xを計算（これが分子になる）
    e_x = (X - max_val).exp()
    denominator = e_x.sum(dim=1, keepdim=True) + 1e-10
    softmax_out = e_x / denominator
    loss = - (y_true * torch.log(softmax_out + 1e-10)).sum() / y_true.shape[0]

    return loss, softmax_out


In [8]:
# tensorの線形結合を返す関数
def linear_comb(X, W, B):
    '''
    X, W, B: torch.tensor
    '''
    return X @ W.T + B

In [9]:
# ReLUの実装
def ReLU(Z):
    '''
    Z: torch.tensor
    '''
    # torch.whereによって要素ごとに条件が真・偽のときで別の値を返せる
    # 下記では0より大きい要素はzの値そのままで、0以下は0.になる。
    return torch.where(Z > 0 , Z, 0.)

順伝播→逆伝播の計算を行う関数を定義。  

In [10]:
# 隠れ層１層の場合のMLP
def forward_and_backward(X, W_list, B_list, y):
    '''
    X: features
    W: List of weights(each edge)
    B: List of Bias(each Layer)
    '''

    #########################
    #### 順伝播（forward）####
    #########################

    # 入力層→隠れ層
    Z1 = linear_comb(X, W_list[0], B_list[0])

    # 活性化関数適用
    A1 = ReLU(Z1)

    # 隠れ層→出力層
    Z2 = linear_comb(A1, W_list[1], B_list[1])

    # 最終出力。損失とソフトマックス関数の出力が返される
    loss, A2 = softmax_cross_entropy(Z2, y)


    #########################
    #### 逆伝播（backward）####
    #########################

    # 最終出力→出力層の出力のbackward。
    # ここでは出力層の活性化関数は恒等関数とし、
    # その出力にsoftmax_cross_entropyを適用した結果をモデルの最終出力として考えている。
    Z2.grad_ = (A2 - y) / X.shape[0]
    # 出力層→隠れ層のbackward
    linear_backward(A1, W_list[1], B_list[1], Z2)
    # 隠れ層の出力側のbackward
    relu_backward(Z1, A1)
    # 隠れ層の入力側のbackward
    linear_backward(X, W_list[0], B_list[0], Z1)
    
    return loss, Z1, A1, Z2, A2
    

### 補足：Z2.grad_をデータ数X.shape[0]で割る理由  
交差エントロピーで算出した全体の損失Lはデータ数で割った平均の形なので、右辺もデータ数で割る必要あり。  
各データに対応する損失をデータ数で割ることで、最終的な全体の損失Lへ与える影響をならしているイメージ。  
誤差逆伝播の講義資料のdL/dZ[2]の式を参照。両辺に1/mが実際はかかっているということ。  

## ■ Autogradの結果と一致することを確認

In [11]:
# 変数定義
learning_rate = 0.03
loss_log = []  #学習時の損失記録用のリスト

In [12]:
# データロード
dataset = datasets.load_digits()
feature_names = dataset['feature_names']
X = torch.tensor(dataset['data'], dtype=torch.float32)
target = torch.tensor(dataset['target'])

# shape確認
print(f'shape of X: {X.shape}')
print(X[1])
print('==========================')
print(f'shape of y_train: {target.shape}')
print(target)

shape of X: torch.Size([1797, 64])
tensor([ 0.,  0.,  0., 12., 13.,  5.,  0.,  0.,  0.,  0.,  0., 11., 16.,  9.,
         0.,  0.,  0.,  0.,  3., 15., 16.,  6.,  0.,  0.,  0.,  7., 15., 16.,
        16.,  2.,  0.,  0.,  0.,  0.,  1., 16., 16.,  3.,  0.,  0.,  0.,  0.,
         1., 16., 16.,  6.,  0.,  0.,  0.,  0.,  1., 16., 16.,  6.,  0.,  0.,
         0.,  0.,  0., 11., 16., 10.,  0.,  0.])
shape of y_train: torch.Size([1797])
tensor([0, 1, 2,  ..., 8, 9, 8])


In [13]:
# 目的変数のエンコーディング
y_true = F.one_hot(target, num_classes=10)
print(f'shape of y_true: {y_true.shape}')

shape of y_true: torch.Size([1797, 10])


In [14]:
# 学習データと検証データを8:2に分ける
X_train, X_val, y_train, y_val = train_test_split(X, y_true, test_size=0.2, random_state=42)

print(f'shape of train data: X_train:{X_train.shape}, y_train:{y_train.shape}')
print(f'shape of validation data: X_val:{X_val.shape}, y_val:{y_val.shape}')

shape of train data: X_train:torch.Size([1437, 64]), y_train:torch.Size([1437, 10])
shape of validation data: X_val:torch.Size([360, 64]), y_val:torch.Size([360, 10])


In [15]:
# 学習データ・検証データの標準化。検証データの標準化には学習データの平均、標準偏差を使用することに注意。  
# 今回のようなデータの場合、全体の平均・標準偏差で標準化するでOK
X_mean = X_train.mean()
X_std = X_train.std()
X_train = (X_train - X_mean) / X_std
X_val = (X_val - X_mean) / X_std

In [16]:
# データ数、特徴量数、隠れ層のノード数、最終的な出力数（ここではクラス数）を定義
m, nf = X_train.shape
nh = 30
n_out = 10

# 入力層→隠れ層の重みW、バイアス項bの初期化
W_1 = torch.randn(size=(nh, nf) ,requires_grad=True) #出力×入力
B_1 = torch.zeros(size=(1, nh), requires_grad=True) # 1 x 出力

# 隠れ層→出力層の重みW、バイアス項bの初期化
W_2 = torch.randn(size=(n_out, nh) ,requires_grad=True) #出力×入力
B_2 = torch.zeros(size=(1, n_out), requires_grad=True) # 1 x 出力

In [17]:
# 予測結果算出
W_list = [W_1, W_2]
B_list = [B_1, B_2]


# y_train_pred = MLP_model(X_train, W_list=W_list, B_list=B_list)
# y_train_pred

### Autogradの結果　

In [30]:
Z1 = linear_comb(X_train, W_list[0], B_list[0])
Z1.retain_grad()

# 活性化関数適用
A1 = ReLU(Z1)
A1.retain_grad()

# 隠れ層→出力層
Z2 = linear_comb(A1, W_list[1], B_list[1])
Z2.retain_grad()

# 最終出力。損失とソフトマックス関数の出力が返される
loss, A2 = softmax_cross_entropy(Z2, y_train)

# lossの勾配を計算
loss.backward()


In [35]:
Z1.grad

tensor([[ 3.6964e-22,  0.0000e+00, -5.9313e-23,  ...,  1.2251e-21,
         -1.7082e-22, -2.0033e-23],
        [ 5.8606e-04,  2.4437e-03, -4.5968e-04,  ...,  5.8667e-04,
         -6.5617e-04, -9.7502e-04],
        [ 7.8489e-04,  0.0000e+00,  3.3980e-04,  ..., -3.6886e-04,
         -1.2971e-03, -1.0156e-04],
        ...,
        [ 1.3819e-19,  0.0000e+00,  5.9020e-19,  ...,  0.0000e+00,
          0.0000e+00,  6.5061e-19],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
         -3.0041e-05,  1.4641e-05],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])

### スクラッチ実装の結果　

In [37]:
loss_auto, Z1_auto, A1_auto, Z2_auto, A2_auto = forward_and_backward(X_train, W_list=W_list, B_list=B_list, y=y_train)

In [44]:
Z1_auto.grad_

tensor([[ 4.6803e-04, -0.0000e+00, -7.5101e-05,  ...,  1.5511e-03,
         -2.1629e-04, -2.5366e-05],
        [ 5.9651e-04,  2.4873e-03, -4.6788e-04,  ...,  5.9713e-04,
         -6.6787e-04, -9.9240e-04],
        [ 7.8570e-04,  0.0000e+00,  3.4015e-04,  ..., -3.6924e-04,
         -1.2984e-03, -1.0167e-04],
        ...,
        [ 1.8919e-04, -0.0000e+00,  8.0803e-04,  ..., -0.0000e+00,
         -0.0000e+00,  8.9073e-04],
        [-0.0000e+00, -0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
         -1.2375e-03,  6.0309e-04],
        [ 3.9760e-04,  0.0000e+00,  0.0000e+00,  ..., -0.0000e+00,
          5.8068e-05,  5.6899e-04]], grad_fn=<MulBackward0>)